## Data Preprocessing: Task 2

In [2]:
#!pip install keras

     |████████████████████████████████| 378kB 1.3MB/s eta 0:00:01
You should consider upgrading via the 'pip install --upgrade pip' command.


In [1]:
##imports go here
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Embedding, LSTM, Bidirectional
from keras.datasets import imdb
import pandas as pd
import spacy
import numpy as np
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize, word_tokenize 
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
nlp = spacy.load('en')
from sklearn.model_selection import train_test_split
import pickle
import numpy as np
import pandas as pd
from keras.models import Model
from keras.layers import Input, Dense, Embedding, SpatialDropout1D, add, concatenate
from keras.layers import CuDNNLSTM, Bidirectional, GlobalMaxPooling1D, GlobalAveragePooling1D
from keras.preprocessing import text, sequence
from gensim.models import KeyedVectors

/Users/chris/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


[nltk_data] Downloading package punkt to /Users/chris/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
review_df=pd.read_csv("yelp_dataset/FinalDataset/reviewOrigFile.csv")
business_df=pd.read_csv("yelp_dataset/FinalDataset/BusinessOrigFile.csv")
usaRestuarants=pd.read_csv("yelp_dataset/FinalDataset/usaRestuarants.csv")

In [3]:
review_df.head()

,business_id,cool,date,funny,review_id,stars,text,useful,user_id
0,ujmEBvifdJM6h6RLv4wQIg,0,2013-05-07 04:34:36,1.0,Q1sbwvVQXV2734tPgoKj4Q,1.0,Total bill for this horrible service? Over $8G...,6.0,hG7b0MtEbXx5QzbzE6C_VA
1,NZnhc2sEQy3RmzKTZnqtwQ,0,2017-01-14 21:30:33,0.0,GJXCdrto3ASJOqKeVWPi6Q,5.0,I *adore* Travis at the Hard Rock's new Kelly ...,0.0,yXQM5uF2jS6es16SJzNHfg
2,WTqjgwHlXbSFevF32_DJVw,0,2016-11-09 20:09:03,0.0,2TzJjDVDEuAW6MR5Vuc1ug,5.0,I have to say that this office really has it t...,3.0,n6-Gk65cPZL6Uz8qRm3NYw
3,ikCg8xy5JIg_NGPx-MSIDA,0,2018-01-09 20:56:38,0.0,yi0R0Ugj_xUx_Nek0-_Qig,5.0,Went in for a lunch. Steak sandwich was delici...,0.0,dacAIZ6fTM6mqwW5uxkskg
4,b1b1eb3uo-w561D0ZfCEiQ,0,2018-01-30 23:07:38,0.0,11a8sVPMUFtaC7_ABRkmtw,1.0,Today was my second out of three sessions I ha...,7.0,ssoyf2_x0EQMed6fgHeMyQ


In [4]:
from datetime import datetime

datetime_object = datetime.strptime('Jun 1 2005  1:33PM', '%b %d %Y %I:%M%p')

Filtering for review year < 2017

In [5]:
#train['Name_length'] = train.Name.apply(len)

In [6]:
review_df['year_date'] = review_df.date.apply(lambda x: x.split("-")[0])

In [7]:
review_df=review_df[review_df['year_date']!="aZs_IrhA7rvuLBumVhJjCw"]

In [8]:
review_df=review_df[review_df['year_date']!="QRn8ELZmvP8S4evnSRU9bQ"]

In [9]:
review_df['year_date'] = pd.to_numeric(review_df["year_date"])

In [10]:
review_df['year_date'].value_counts()

2017    1217673
2018    1177662
2016    1098786
2015     952400
2014     704862
2013     491294
2012     367090
2011     302867
2010     187387
2009     101173
2008      57347
2007      21389
2006       5081
2005        876
2004         13
Name: year_date, dtype: int64

In [11]:
len(review_df)

6685900

In [12]:
review_df_year=review_df[review_df["year_date"]<2017]
len(review_df_year)

4290565

In [10]:
review_df_filtered = review_df_year[review_df_year.business_id.isin(usaRestuarants.business_id)]

In [11]:
len(review_df_filtered)

2231445

In [12]:
review_df_filtered=review_df_filtered.dropna()

In [29]:
len(review_df_filtered)

2231443

In [18]:
review_df_filtered.reset_index(inplace = True) 

In [19]:
review_df_filtered.head()

,index,business_id,cool,date,funny,review_id,stars,text,useful,user_id,year_date,cleaned_text
0,5,eU_713ec6fTGNO4BegRaww,0,2013-01-20 13:25:59,0.0,fdiNeiN_hoCxCMy2wTRW9g,4.0,I'll be the first to admit that I was not exci...,0.0,w31MKYsNFMrjhWxxAb5wIw,2013,
1,7,zvO-PJCpNk4fgAVUnExYAA,1,2010-10-05 19:12:35,1.0,8e9HxxLjjqc9ez5ezzN7iQ,1.0,This place has gone down hill. Clearly they h...,3.0,d6xvYpyzcfbF_AZ8vMB7QA,2010,
2,10,8mIrX_LrOnAqWsB5JrOojQ,0,2011-11-30 02:11:15,0.0,kbtscdyz6lvrtGjD1quQTg,4.0,"Like walking back in time, every Saturday morn...",0.0,FIk4lQQu1eTe2EpzQ4xhBA,2011,
3,12,FxLfqxdYPA6Z85PFKaqLrg,0,2016-05-07 01:36:53,0.0,Z7wgXp98wYB57QdRY3HQ3w,4.0,Wow. So surprised at the one and two star revi...,0.0,GYNnVehQeXjty0xH7-6Fhw,2016,
4,16,NyLYY8q1-H3hfsTwuwLPCg,0,2015-01-03 22:47:34,0.0,1wVA2-vQIuW_ClmXkDxqMQ,4.0,Great lunch today. Staff was very helpful in a...,0.0,86J5DwcFk4f4In1Vxe2TvA,2015,


# Cleaning and preprocessing data

In [20]:
def clean(s):
    string = nlp(s)
    words = [token.text for token in string
         if token.is_stop != True and token.is_punct != True and token.is_alpha == True]
    return words

In [22]:
temp= review_df_filtered[:20]
temp['cleaned_text'] = temp.text.apply(clean)

/Users/chris/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [30]:
#downsampling rows with 0 votes randomly:
#reviews_USA["class_useful"]=np.where(reviews_USA["useful"]==0,1,0)
review_df_filtered["class_useful"]=np.where(review_df_filtered["useful"]==0,0,1)

In [28]:
review_df_filtered["useful"].value_counts()

0.0      1148262
1.0       513166
2.0       242990
3.0       123281
4.0        67638
5.0        40373
6.0        25249
7.0        16872
8.0        11813
9.0         8406
10.0        6341
11.0        4621
12.0        3766
13.0        2988
14.0        2297
15.0        1814
16.0        1582
17.0        1261
18.0        1067
19.0         917
20.0         746
21.0         587
22.0         526
23.0         475
25.0         395
24.0         389
26.0         329
28.0         265
27.0         260
29.0         223
          ...   
507.0          1
125.0          1
110.0          1
120.0          1
155.0          1
156.0          1
158.0          1
163.0          1
217.0          1
216.0          1
213.0          1
210.0          1
207.0          1
204.0          1
202.0          1
114.0          1
196.0          1
194.0          1
193.0          1
191.0          1
188.0          1
184.0          1
182.0          1
180.0          1
175.0          1
172.0          1
168.0          1
165.0         

In [31]:
review_df_filtered["class_useful"].value_counts()

0    1148262
1    1083181
Name: class_useful, dtype: int64

## Downsampling Data based on class of usefulness

In [49]:
from collections import Counter
from sklearn.datasets import make_classification
from imblearn.under_sampling import RandomUnderSampler
X, y = make_classification(n_classes=2, class_sep=2,weights=[0.5, 0.5], n_informative=3, n_redundant=1, flip_y=0,
n_features=20, n_clusters_per_class=1, n_samples=1000, random_state=10)
print('Original dataset shape %s' % Counter(y))

rus = RandomUnderSampler(random_state=42,ratio={1: 200000, 0: 200000})
reviews_resampled, reviews_class_resampled = rus.fit_resample(review_df_filtered, np.array(review_df_filtered["class_useful"]))
print('Resampled dataset shape %s' % Counter(reviews_class_resampled))

Original dataset shape Counter({0: 500, 1: 500})
Resampled dataset shape Counter({0: 200000, 1: 200000})


In [50]:
ln = [i for i in review_df_filtered.columns]
ln

['index',
 'business_id',
 'cool',
 'date',
 'funny',
 'review_id',
 'stars',
 'text',
 'useful',
 'user_id',
 'year_date',
 'cleaned_text',
 'class_useful']

In [51]:
reviews_resampled = pd.DataFrame(reviews_resampled,columns = ln)

In [53]:
reviews_resampled.head()

,index,business_id,cool,date,funny,review_id,stars,text,useful,user_id,year_date,cleaned_text,class_useful
0,529146,5ma1OUQWy_Ds80xUiP7JPQ,0,2016-03-26 23:54:48,0,eRZpzG1i6rPzk57KLtFtLg,4,the only reason I am giving it a 4 star becaus...,0,pEKFes5cKIO4kl1tqqUy-g,2016,"[reason, I, giving, star, Katsu, chicken, good...",0
1,1488069,lG759ODx7tj6wbS7rYip1g,0,2016-04-27 04:21:50,0,RNnvqCBtp3Hj4JoLzgFewQ,5,"Wow killer good food, but environment basic. T...",0,Ed8hLEXNsIybzH_gtXvwjA,2016,"[Wow, killer, good, food, environment, basic, ...",0
2,6121086,6kD0vTLj3oZJm7kXRSvfKw,0,2015-10-09 07:44:51,0,zlZn5M9mDwtGzH7trtDjWg,4,Eggplant parmesan and the italian wedding soup...,0,srho-TU4JN2PN3g8oXaoEg,2015,"[Eggplant, parmesan, italian, wedding, soup, g...",0
3,6018658,zXBrfL_ohYMuBZWvF8EcSA,2,2016-02-10 03:03:32,0,CNluDuMEU7ghDNr8X3u-yQ,4,"Tried this place for lunch, although busy and ...",0,KCD6daAp293FoOUoTjT_YA,2016,"[Tried, place, lunch, busy, little, noisy, foo...",0
4,1521119,Z1E36owKeeZKDL-9ZuanJw,0,2016-01-08 05:08:20,1,tnk7dYZSVG3-QMIwInydlQ,4,Food was definitely tasty. I tried the baby ba...,0,c6lAB08lDzAVF55zYn7nXA,2016,"[Food, definitely, tasty, I, tried, baby, ribs...",0


In [54]:
for i in range(200000,len(reviews_resampled),10000):
    print(i,i+10000)
    reviews_resampled.iloc[i:i+10000,-2] = reviews_resampled.iloc[i:i+10000,-6].apply(clean)


200000 210000
210000 220000
220000 230000
230000 240000
240000 250000
250000 260000
260000 270000
270000 280000
280000 290000
290000 300000
300000 310000
310000 320000


KeyboardInterrupt: 

In [55]:
for i in range(310000,len(reviews_resampled),10000):
    print(i,i+10000)
    reviews_resampled.iloc[i:i+10000,-2] = reviews_resampled.iloc[i:i+10000,-6].apply(clean)

310000 320000
320000 330000
330000 340000
340000 350000
350000 360000
360000 370000
370000 380000
380000 390000
390000 400000


In [59]:
#reviews_balanced['len_tokens'] = 0
reviews_resampled['reviews_length'] = reviews_resampled.cleaned_text.apply(len)

In [62]:
len(reviews_resampled)

400000

In [64]:
reviews_resampled.head(-5)

,index,business_id,cool,date,funny,review_id,stars,text,useful,user_id,year_date,cleaned_text,class_useful,reviews_length
0,529146,5ma1OUQWy_Ds80xUiP7JPQ,0,2016-03-26 23:54:48,0,eRZpzG1i6rPzk57KLtFtLg,4,the only reason I am giving it a 4 star becaus...,0,pEKFes5cKIO4kl1tqqUy-g,2016,"[reason, I, giving, star, Katsu, chicken, good...",0,34
1,1488069,lG759ODx7tj6wbS7rYip1g,0,2016-04-27 04:21:50,0,RNnvqCBtp3Hj4JoLzgFewQ,5,"Wow killer good food, but environment basic. T...",0,Ed8hLEXNsIybzH_gtXvwjA,2016,"[Wow, killer, good, food, environment, basic, ...",0,27
2,6121086,6kD0vTLj3oZJm7kXRSvfKw,0,2015-10-09 07:44:51,0,zlZn5M9mDwtGzH7trtDjWg,4,Eggplant parmesan and the italian wedding soup...,0,srho-TU4JN2PN3g8oXaoEg,2015,"[Eggplant, parmesan, italian, wedding, soup, g...",0,24
3,6018658,zXBrfL_ohYMuBZWvF8EcSA,2,2016-02-10 03:03:32,0,CNluDuMEU7ghDNr8X3u-yQ,4,"Tried this place for lunch, although busy and ...",0,KCD6daAp293FoOUoTjT_YA,2016,"[Tried, place, lunch, busy, little, noisy, foo...",0,43
4,1521119,Z1E36owKeeZKDL-9ZuanJw,0,2016-01-08 05:08:20,1,tnk7dYZSVG3-QMIwInydlQ,4,Food was definitely tasty. I tried the baby ba...,0,c6lAB08lDzAVF55zYn7nXA,2016,"[Food, definitely, tasty, I, tried, baby, ribs...",0,62
5,5132398,CBgcKcrbN7VSoofxCsCOBA,0,2014-07-22 01:59:40,0,vzqoGoQ3NFcX-kKZsbcyAA,5,One of the best butcher shops I have ever been...,0,trGiQWWedQv70yaNr3mORQ,2014,"[One, best, butcher, shops, I, The, owner, ext...",0,21
6,6328475,DXP3MbE8srQITLlAuS45aQ,0,2015-03-24 00:43:06,0,pUKNABz7cvy-YrTo99fbjw,4,"Had hot pot, dishes and Ma La Tang (a spicy bo...",0,49GlLnhi2d8YbbR6tzDFUQ,2015,"[Had, hot, pot, dishes, Ma, La, Tang, spicy, b...",0,106
7,5581519,Zmfq3DVhHdtVChhSxzoUHw,0,2014-08-29 19:51:18,0,xh1NgDwGQx4Sl6P4_2IJGA,5,Hand's down the BEST Chicken Pad Thai I have e...,0,pomBxVCy2noxvpiUCXQK1Q,2014,"[Hand, BEST, Chicken, Pad, Thai, I, eaten, Eve...",0,12
8,1525272,-8QlV3b_9H4BAh6LgMIr1g,0,2015-03-22 00:47:40,0,_ojrq4WHar1u67gvDgpA9g,5,"Food is terrific, reasonably priced, lots of v...",0,VO0s6iWye7D51KcO3BnhlA,2015,"[Food, terrific, reasonably, priced, lots, veg...",0,24
9,170075,KUbCQ2ah__c5x0q_6b4oTA,0,2012-07-23 01:59:54,0,yROAMx1qgeF1EeG8pfmb3w,5,Awesome. No question I could work in PR for th...,0,QFYT4jsU_MYEjBIhvHNQGw,2012,"[Awesome, No, question, I, work, PR, spot, Fre...",0,19


In [66]:
reviews_resampled.class_useful.value_counts()

1    200000
0    200000
Name: class_useful, dtype: int64

## Removing reviews with length < 5

In [89]:
len(reviews_resampled[reviews_resampled["reviews_length"]>=5])

399649

In [90]:
reviews_resampled_filtered=reviews_resampled[reviews_resampled["reviews_length"]>=5]

In [91]:
reviews_resampled_filtered.class_useful.value_counts()

1    199877
0    199772
Name: class_useful, dtype: int64

In [92]:
reviews_resampled_filtered.head()

,index,business_id,cool,date,funny,review_id,stars,text,useful,user_id,year_date,cleaned_text,class_useful,reviews_length
0,529146,5ma1OUQWy_Ds80xUiP7JPQ,0,2016-03-26 23:54:48,0,eRZpzG1i6rPzk57KLtFtLg,4,the only reason I am giving it a 4 star becaus...,0,pEKFes5cKIO4kl1tqqUy-g,2016,"[reason, I, giving, star, Katsu, chicken, good...",0,34
1,1488069,lG759ODx7tj6wbS7rYip1g,0,2016-04-27 04:21:50,0,RNnvqCBtp3Hj4JoLzgFewQ,5,"Wow killer good food, but environment basic. T...",0,Ed8hLEXNsIybzH_gtXvwjA,2016,"[Wow, killer, good, food, environment, basic, ...",0,27
2,6121086,6kD0vTLj3oZJm7kXRSvfKw,0,2015-10-09 07:44:51,0,zlZn5M9mDwtGzH7trtDjWg,4,Eggplant parmesan and the italian wedding soup...,0,srho-TU4JN2PN3g8oXaoEg,2015,"[Eggplant, parmesan, italian, wedding, soup, g...",0,24
3,6018658,zXBrfL_ohYMuBZWvF8EcSA,2,2016-02-10 03:03:32,0,CNluDuMEU7ghDNr8X3u-yQ,4,"Tried this place for lunch, although busy and ...",0,KCD6daAp293FoOUoTjT_YA,2016,"[Tried, place, lunch, busy, little, noisy, foo...",0,43
4,1521119,Z1E36owKeeZKDL-9ZuanJw,0,2016-01-08 05:08:20,1,tnk7dYZSVG3-QMIwInydlQ,4,Food was definitely tasty. I tried the baby ba...,0,c6lAB08lDzAVF55zYn7nXA,2016,"[Food, definitely, tasty, I, tried, baby, ribs...",0,62


In [93]:
reviews_resampled_filtered["cleaned_text_final"] = reviews_resampled_filtered["cleaned_text"].str.join(" ") 

/Users/chris/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [94]:
reviews_resampled_filtered.head()

,index,business_id,cool,date,funny,review_id,stars,text,useful,user_id,year_date,cleaned_text,class_useful,reviews_length,cleaned_text_final
0,529146,5ma1OUQWy_Ds80xUiP7JPQ,0,2016-03-26 23:54:48,0,eRZpzG1i6rPzk57KLtFtLg,4,the only reason I am giving it a 4 star becaus...,0,pEKFes5cKIO4kl1tqqUy-g,2016,"[reason, I, giving, star, Katsu, chicken, good...",0,34,reason I giving star Katsu chicken good flavor...
1,1488069,lG759ODx7tj6wbS7rYip1g,0,2016-04-27 04:21:50,0,RNnvqCBtp3Hj4JoLzgFewQ,5,"Wow killer good food, but environment basic. T...",0,Ed8hLEXNsIybzH_gtXvwjA,2016,"[Wow, killer, good, food, environment, basic, ...",0,27,Wow killer good food environment basic They im...
2,6121086,6kD0vTLj3oZJm7kXRSvfKw,0,2015-10-09 07:44:51,0,zlZn5M9mDwtGzH7trtDjWg,4,Eggplant parmesan and the italian wedding soup...,0,srho-TU4JN2PN3g8oXaoEg,2015,"[Eggplant, parmesan, italian, wedding, soup, g...",0,24,Eggplant parmesan italian wedding soup great T...
3,6018658,zXBrfL_ohYMuBZWvF8EcSA,2,2016-02-10 03:03:32,0,CNluDuMEU7ghDNr8X3u-yQ,4,"Tried this place for lunch, although busy and ...",0,KCD6daAp293FoOUoTjT_YA,2016,"[Tried, place, lunch, busy, little, noisy, foo...",0,43,Tried place lunch busy little noisy food makes...
4,1521119,Z1E36owKeeZKDL-9ZuanJw,0,2016-01-08 05:08:20,1,tnk7dYZSVG3-QMIwInydlQ,4,Food was definitely tasty. I tried the baby ba...,0,c6lAB08lDzAVF55zYn7nXA,2016,"[Food, definitely, tasty, I, tried, baby, ribs...",0,62,Food definitely tasty I tried baby ribs links ...


In [96]:
reviews_final_df = reviews_resampled_filtered.loc[:, ["business_id","user_id","review_id",'cleaned_text_final',"stars",'useful',"class_useful","year_date"]]

reviews_final_df.head()


,business_id,user_id,review_id,cleaned_text_final,stars,useful,class_useful,year_date
0,5ma1OUQWy_Ds80xUiP7JPQ,pEKFes5cKIO4kl1tqqUy-g,eRZpzG1i6rPzk57KLtFtLg,reason I giving star Katsu chicken good flavor...,4,0,0,2016
1,lG759ODx7tj6wbS7rYip1g,Ed8hLEXNsIybzH_gtXvwjA,RNnvqCBtp3Hj4JoLzgFewQ,Wow killer good food environment basic They im...,5,0,0,2016
2,6kD0vTLj3oZJm7kXRSvfKw,srho-TU4JN2PN3g8oXaoEg,zlZn5M9mDwtGzH7trtDjWg,Eggplant parmesan italian wedding soup great T...,4,0,0,2015
3,zXBrfL_ohYMuBZWvF8EcSA,KCD6daAp293FoOUoTjT_YA,CNluDuMEU7ghDNr8X3u-yQ,Tried place lunch busy little noisy food makes...,4,0,0,2016
4,Z1E36owKeeZKDL-9ZuanJw,c6lAB08lDzAVF55zYn7nXA,tnk7dYZSVG3-QMIwInydlQ,Food definitely tasty I tried baby ribs links ...,4,0,0,2016


In [99]:
with open("reviews_final_df_fulldata.pkl", 'wb') as f:
    pickle.dump(reviews_final_df, f)

In [100]:
with open("reviews_all_columns_fulldata.pkl", 'wb') as f:
    pickle.dump(reviews_resampled_filtered, f)

In [101]:
reviews_final_df.to_csv("reviews_final_df_fulldata.csv",index=False)